In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
from holoviews import opts
import panel as pn
import panel.widgets as pnw
pn.extension()

import util

/glade/work/mclong/miniconda3/envs/_unison/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
catalog = util.to_intake_esm()
catalog = catalog.search(name='ann-timeseries')

In [7]:
df_catalog = catalog.df
df_catalog

,component,experiment,stream,asset,catalog_file,key,member_id,name,variable
0,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,1,ann-timeseries,POC_FLUX_100m
1,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,2,ann-timeseries,POC_FLUX_100m
2,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,3,ann-timeseries,POC_FLUX_100m
3,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,4,ann-timeseries,POC_FLUX_100m
4,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,5,ann-timeseries,POC_FLUX_100m
5,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,6,ann-timeseries,POC_FLUX_100m
6,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,7,ann-timeseries,POC_FLUX_100m
7,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,8,ann-timeseries,POC_FLUX_100m
8,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,9,ann-timeseries,POC_FLUX_100m
9,ocn,historical,pop.h,assets/campaign-cesm2-cmip6-timeseries.ocn.his...,data/campaign-cesm2-cmip6-timeseries.json,ocn.historical.pop.h,10,ann-timeseries,POC_FLUX_100m


In [9]:
regions_list = ['Global', 'Atlantic Ocean', 'Pacific Ocean', 'Indian Ocean']
variable_list = list(df_catalog.variable.unique())
member_id_list = list(df_catalog.member_id.unique())

case = pnw.MultiSelect(
    name="Member", value=member_id_list[:1], options=member_id_list, height=150
)
variable = pnw.Select(name="Variable", value=variable_list[0], options=variable_list)
region = pnw.Select(name="Region", value=regions_list[0], options=regions_list)


@pn.depends(case.param.value, variable.param.value, region.param.value)
def create_figure(cases, variable, region):
    if not cases:
        return
    dfs = []
    for case in cases:
        path = df_catalog.loc[
            (df_catalog.member_id == case) & (df_catalog.variable == variable)
        ].asset.values[0]
        dfi = xr.open_zarr(path)[variable].sel(region=region, drop=True).to_dataframe()
        dfi["case"] = case
        dfs.append(dfi)
    df_data = pd.concat(dfs)

    vdim = hv.Dimension(variable)
    ds = hv.Dataset(df_data, vdims=vdim)
    
    curves = ds.to(hv.Curve, "time", groupby="case").overlay()

    return curves.opts(
        opts.Curve(
            show_frame=False,
            tools=["hover"],
            title=f"{region}: {variable}",
            width=600,
            height=400,
        )
    )

widgets = pn.WidgetBox(case, variable, region, width=200, height=400)
pn.Row(widgets, create_figure).servable("Cross-selector")

Row
    [0] WidgetBox(height=400, sizing_mode='fixed', width=200)
        [0] MultiSelect(height=150, name='Member', options=[1, 2, 3, 4, 5, ...], value=[1])
        [1] Select(name='Variable', options=['POC_FLUX_100m'], value='POC_FLUX_100m')
        [2] Select(name='Region', options=['Global', 'Atlantic Ocean...], value='Global')
    [1] ParamFunction(function)